# Exercise: Neural Network 

In todays exercise you will make a simple feedforward neural network that can replicate the *in silico* color mixer. You will be given quite a lot of code and only have to fill in a few blanks for this notebook to work. However, the purpose of this exercise is not only getting a NN that works but for you to gain some insight into neural networks and experiment with varying various aspects. You can vary 

* The noise level in the SilicoColorMixer used to generate data
* The input data
  * Amount of data
  * Different methods to pretreat data
  * Combine data with different pretreatment in one data set
* The Neural Network architecture
  * Number of hidden layers
  * Size of hidden layers
  * Loss function
* Evaluation method
* Any other interesting parameter/feature that you might come up with

Start by getting the MLPRegressor from scikit-learn.

In [ ]:
try:
    from sklearn.neural_network import MLPRegressor
except:
    !pip3 install scikit-learn --user --upgrade
    from sklearn.neural_network import MLPRegressor

Load in other packages that you (might) need.

In [ ]:
import numpy as np
from IPython import display
import matplotlib.pyplot as plt
from plot_pie_charts import make_piechart_plot
from silico_color_mixer import SilicoColorMixer

Now generate som input data. You can generate data in various ways. You can specify color inputs that you would really like to be in the data set or do as done below - generate some random data. We generate a 400 by 4 array with random numbers between 0 and 1. 

In [ ]:
x = np.array(np.random.rand(400, 4))

As nice trick when dealing with NN is to normalize your inputs. It is even more true in this case where our true model, the `SilicoColorMixer`, only cares about the ratio of inputs as it normalizes within. Below is a function that normalizes a color list. The `try - except` clause makes the function less sensitive to the format of the input data thus gives some flexibility during other data pretreatments. 

In [ ]:
def normalize(color_list):
    sum_list = sum(color_list)
    try:
        norm_list = [1. / sum_list[0] * i for i in color_list]
    except:
        norm_list = [1. / sum_list * i for i in color_list]
    return norm_list

Now pretreat the data. To avoid all data being a mix of all colors, you could remove one color from each data point at random as done below. You can do this two times if you like or come up with your own way of pretreating data to get a different training set.

In [ ]:
for idx, line in enumerate(x):
    line[np.random.randint(0,4)] = 0
    x[idx] = normalize(line)

If you want to combine two lists which have undergone different data treatment you can use

In [ ]:
z = np.array(np.random.rand(100, 4))
xz = np.concatenate((x, z))
print(len(xz)) # The length of the new array

Now, make a list with the colors that are generated by the inputs. Our training data consist of input and corresponding output. What type of learning is this?

In [ ]:
rgbs = [] 

Initially try using a color mixer without noise. Once you get the grasp of it, go back to here and add some noise.

In [ ]:
data_generator_mixer = SilicoColorMixer(noise=False)
# data_generator_mixer = SilicoColorMixer(noise={'colors': 1, 'volume': 0.02, 'measurement': 2},)

Generate the output.

Note: We use a lot of `for` loops to the point where professional programmers will likely cry out in agony, because they are slow compared to smarter ways of obtaining the same operations. We use them because they are easy to write, read, and understand.

In [ ]:
for rand_data in x:
    rgbs.append(data_generator_mixer.run_cuvette(rand_data))

In [ ]:
print(len(rgbs))

Try to visualize the colors in your training data

In [ ]:
#print(rgbs)
plt.axis('off')
data_array = np.asarray(rgbs).reshape(20, 20, 3)  # Change this as needed, 20 x 20 = 400
plt.imshow(np.asarray(data_array, dtype=np.uint8))
plt.imshow(np.array(data_array, dtype=np.uint8))

See if you can verify that you have a good distribution of inputs.

Change the format of the output from tuple to list, which is what you need to give the neural network (Programmers, look away!)

In [ ]:
rgbs_list = []
for color in rgbs:
    rgbs_list.append(list(color))

Now you are ready to train a neural network. We are using the "Regressor" because we want numerical input to return numerical output. Seek out [the online documentation of MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html). Figure out what the different keywords mean and what other keywords can be specified.

Below, a neural network is initialized and then trained to your data with the `fit` method. If training takes more than 15 seconds, you have overdone it in some way or another.

In [ ]:
mpl = MLPRegressor(solver='lbfgs', alpha=1e-5,
                   hidden_layer_sizes=(10,10, 3), random_state=1, max_iter=4000)

mpl.fit(x,rgbs_list)

Now, test how well the neural network performs. Use a noiseless color mixer for this purpose.

In [ ]:
test_mixer = SilicoColorMixer(noise=False)

You can compare point by point. This can be good sometimes if you have certain points that you know your are particularly interested in. You could also have included such points in the training set

In [ ]:
point = [0.25, 0.25, 0., 0.5]
print(mpl.predict([point]))
print(test_mixer.run_cuvette(point))

You can do the performance evaluation on a larger data set and more systematically. Generate some data the way you did before. Note, that when you generate data the same way you may inadvertently sample the same subset of data that your used to train the NN model.

In [ ]:
x_test = np.array(np.random.rand(100, 4))
for idx, line in enumerate(x_test):
    line[np.random.randint(0,4)] = 0
    x_test[idx] = normalize(line)

A good way to quantify the difference between the test_mixer and the NN is to use your good old "score" function. Copy it in the cell below. Note that the output from the NN will be a list. Your "score" function might treat lists and tuples the same (`input_color1[0]` does not care whether `input_color1` is a list or a tuple.

In [ ]:
# Your "score" function here

Calculate the difference scores for the points in the test set and add them to a list. 

In [ ]:
scores = []
for x_test_point in x_test:
    nn = mpl.predict([x_test_point])
    silico = test_mixer.run_cuvette(x_test_point)  
    scores.append(your_score_function(nn[0], silico))  # Replace with your score function here

In [ ]:
print(scores)

We can reduce this to a few interesting numbers by calculating statistics.

In [ ]:
print("Mean: ",np.mean(scores)," Standard deviation: ", np.std(scores))

Try to rerun the evaluation without changing code but using a new set of random test data. How much does it change? What if you use new training data and retrain the NN. How much do the numbers change?

Rinse and repeat. 


Once you have a good understanding and have examined how varying different parameters change the result, you are done with the exercise. Take a moment to appreciate yourself for your efforts.